## Установка модулей

In [407]:
%%capture
!pip install -q langchain langchain-gigachat langchain-core

## Импорты и настройка

In [408]:
import json
import uuid
from datetime import datetime
from typing import List, Dict, Optional
from dataclasses import dataclass, asdict

from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from google.colab import files
from google.colab import userdata

GIGACHAT_API_KEY = userdata.get("GIGACHAT_API_KEY")
AI_TEMPERATURE = 0.7
MAX_TOKENS = 2000

## Структуры данных

In [409]:
@dataclass
class Turn:
    turn_id: int
    agent_visible_message: str
    user_message: str
    internal_thoughts: str

@dataclass
class FinalFeedback:
    grade: str
    hiring_recommendation: str
    confidence_score: int
    confirmed_skills: List[str]
    knowledge_gaps: List[Dict[str, str]]
    soft_skills: Dict[str, str]
    roadmap: List[str]

    def print_feedback(self):
        print(f"Грейд: {self.grade}")
        print(f"Рекомендация: {self.hiring_recommendation}")
        print(f"Уверенность в результате: {self.confidence_score}%")
        print(f"\nПодтвержденные навыки: {', '.join(self.confirmed_skills)}")

        print("\nПробелы в знаниях:")
        for gap in self.knowledge_gaps:
            print(f"  - {gap.get('topic')}: {gap.get('correct_answer')}")

        if self.soft_skills:
            for skill, comment in self.soft_skills.items():
                print(f"{skill}: {comment}")

        print("\nДорожная карта обучения:")
        for step, topic in enumerate(self.roadmap, 1):
            print(f"  {step}. {topic}")



## GigaChat клиент

In [410]:
class GigaChatClient:
    def __init__(self, api_key: str, scope: str = "GIGACHAT_API_PERS"):
        self.llm = GigaChat(
            credentials=api_key,
            scope=scope,
            verify_ssl_certs=False,
            temperature=AI_TEMPERATURE,
            max_tokens=MAX_TOKENS
        )

    def get_llm(self, temperature: Optional[float] = None, max_tokens: Optional[int] = None):
        if temperature is not None or max_tokens is not None:
            return GigaChat(
                credentials=self.llm.credentials,
                scope=self.llm.scope,
                verify_ssl_certs=False,
                temperature=temperature or self.llm.temperature,
                max_tokens=max_tokens or self.llm.max_tokens,
                model="GigaChat-Pro"
            )
        return self.llm

## Агент-Наблюдатель


In [411]:
class ObserverAgent:
    def __init__(self, client: GigaChatClient):
        self.client = client

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """
Ты — Наблюдатель на техническом интервью.

Твоя задача:
1. Анализировать ответы кандидата на технические вопросы
2. Выявлять признаки неуверенности, некомпетентности или попыток обмана
3. Определять галлюцинации и фейковые факты
4. Давать рекомендации Интервьюеру о следующих шагах
5. Отслеживать попытки съехать с темы и возвращать диалог в нужное русло

Если кандидат выдает явный бред, то в поле "recommendation" ты обязательно должен написать: "Обязательно указать на ошибку, затем вернуться к теме"
Если ошибок не было, но кандидат задал вопрос, то в поле "recommendation" ты обязательно должен написать: "обязательно кратко ответь на вопрос и вернись к теме"

Отвечай строго в формате JSON (без markdown разметки):
{{
    "analysis": "Твой анализ ответа кандидата",
    "detected_issues": ["список проблем"],
    "knowledge_level": "оценка уровня знаний (high/medium/low)",
    "is_hallucination": true/false,
    "is_off_topic": true/false,
    "recommendation": "что должен сделать интервьюер"
}}"""),
            ("human", """Предыдущий контекст:
{context}

Текущий вопрос: {question}
Ответ кандидата: {answer}

Проанализируй ответ и дай рекомендацию о следующем шаге.""")
        ])

        self.chain = self.prompt | self.client.get_llm() | StrOutputParser()

    def analyze(self, question: str, answer: str, context: List[Turn]) -> Dict:
        context_str = "\n".join([
            f"Q: {t.agent_visible_message}\nA: {t.user_message}"
            for t in context[-3:]
        ]) if context else "Нет предыдущего контекста"

        try:
            response = self.chain.invoke({
                "context": context_str,
                "question": question,
                "answer": answer
            })

            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response

            return json.loads(json_str)
        except Exception as e:
            print(f"Ошибка парсинга Observer: {e}")
            return {
                "analysis": response if 'response' in locals() else "Ошибка анализа",
                "detected_issues": [],
                "knowledge_level": "medium",
                "is_hallucination": False,
                "is_off_topic": False,
                "recommendation": "Продолжить диалог"
            }

## Агент-Интервьюер


In [412]:
class InterviewerAgent:
    def __init__(self, client: GigaChatClient):
        self.client = client
        self.current_difficulty = "junior"

    def get_system_prompt(self, position: str, grade: str, experience: str) -> str:
        return f"""
Ты — Профессиональный Технический Интервьюер.

Информация о кандидате:
- Позиция: {position}
- Целевой грейд: {grade}
- Опыт: {experience}

Твои обязанности:
1. Проводить техническое интервью, задавая релевантные вопросы
2. Адаптировать сложность вопросов на основе ответов
3. Не задавать вопросы, на которые кандидат уже ответил
4. Слушай рекомендации от Observer

Правила:
- Не повторяй вопросы, если кандидат уже давал на них ответ
- Если кандидат выдумывает факты, вежливо укажи на ошибку
- Если кандидат уходит от темы, верни беседу обратно
- Если кандидат задает вопрос, ответь на него кратко и задай следующий технический вопрос
- Текущий уровень сложности вопросов: {self.current_difficulty}

ВАЖНО: Сначала в блоке <reasoning> изложи свои рассуждения о том, какой вопрос задать и почему.
Затем в блоке <response> дай финальный ответ кандидату.

Формат ответа:
<reasoning>
Твои внутренние рассуждения: анализ ситуации, выбор следующего вопроса, стратегия
</reasoning>
<response>
Текст сообщения для кандидата
</response>"""

    def generate_question(self, position: str, grade: str, experience: str,
                         observer_recommendation: str, context: List[Turn],
                         current_answer: str = None) -> tuple[str, str]:
        context_str = "\n".join([
            f"Интервьюер: {t.agent_visible_message}\nКандидат: {t.user_message}"
            for t in context[-5:]
        ]) if context else "Начало интервью"

        current_interaction = ""
        if current_answer:
            current_interaction = f"\nПоследний ответ кандидата: {current_answer}"

        user_message = f"""
Предыдущий диалог:
{context_str}
{current_interaction}

Рекомендация от Observer: {observer_recommendation}

Обязательно выполни рекомендацию от обсервера. Продолжай выполнять интервью"""

        prompt = ChatPromptTemplate.from_messages([
            ("system", self.get_system_prompt(position, grade, experience)),
            ("human", "{user_message}")
        ])

        chain = prompt | self.client.get_llm(temperature=0.8) | StrOutputParser()

        full_response = chain.invoke({"user_message": user_message})

        reasoning = ""
        response = full_response

        if "<reasoning>" in full_response and "</reasoning>" in full_response:
            reasoning_start = full_response.find("<reasoning>") + len("<reasoning>")
            reasoning_end = full_response.find("</reasoning>")
            reasoning = full_response[reasoning_start:reasoning_end].strip()

        if "<response>" in full_response and "</response>" in full_response:
            response_start = full_response.find("<response>") + len("<response>")
            response_end = full_response.find("</response>")
            response = full_response[response_start:response_end].strip()
        elif "<reasoning>" in full_response:
            response = full_response.split("</reasoning>")[-1].strip()

        return response, reasoning

    def adjust_difficulty(self, knowledge_level: str):
        if knowledge_level == "high":
            if self.current_difficulty == "junior":
                self.current_difficulty = "middle"
            elif self.current_difficulty == "middle":
                self.current_difficulty = "senior"
        elif knowledge_level == "low":
            if self.current_difficulty == "senior":
                self.current_difficulty = "middle"
            elif self.current_difficulty == "middle":
                self.current_difficulty = "junior"


## Агент-Менеджер


In [413]:
class ManagerAgent:

    def __init__(self, client: GigaChatClient):
        self.client = client

    def generate_feedback(self, position: str, grade: str,
                         experience: str, turns: List[Turn]) -> FinalFeedback:

        dialog = "\n".join([
            f"Q{t.turn_id}: {t.agent_visible_message}\nA{t.turn_id}: {t.user_message}\nThoughts: {t.internal_thoughts}"
            for t in turns
        ])

        prompt = ChatPromptTemplate.from_messages([
            ("system", f"""Ты — HR Manager, принимающий финальное решение о найме.

Позиция: {position}
Ожидаемый грейд: {grade}
Заявленный опыт: {experience}

КРИТИЧЕСКИ ВАЖНО ПРИ ОЦЕНКЕ НАВЫКОВ:
1. В список "confirmed_skills" добавляй ТОЛЬКО те навыки, по которым кандидат дал ПРАВИЛЬНЫЙ ТЕХНИЧЕСКИЙ ОТВЕТ.
2. Если кандидат ушел от ответа, "заболтал" тему или сказал глупость (например, про Python 4.0) — ЭТО НЕ НАВЫК, это пробел.
3. Если кандидат просто заявил "Я знаю SQL", но не ответил на вопрос по SQL — навык НЕ подтвержден.
4. Если были замечены галлюцинации (фактические ошибки) — обязательно отрази это в "soft_skills" (честность/адекватность) и снизь грейд.

Проанализируй полное интервью и создай структурированный фидбэк в формате JSON (без markdown разметки).
Структура JSON:
{{{{
    "grade": "Junior/Middle/Senior",
    "hiring_recommendation": "Hire/No Hire/Strong Hire",
    "confidence_score": число от 0 до 100 насколько ты уверен в правильности своего вердикта,
    "confirmed_skills": ["навык1", "навык2"],
    "knowledge_gaps": [
        {{{{"topic": "проблемная тема", "correct_answer": "правильный ответ"}}}}
    ],
    "soft_skills": {{{{
        "clarity": "оценка ясности изложения",
        "honesty": "оценка честности",
        "engagement": "оценка вовлеченности"
    }}}},
    "roadmap": ["тема для изучения 1", "тема 2"]
}}}}

Будь объективным и конструктивным."""),
            ("human", """Полный диалог интервью:

{dialog}

Создай финальный фидбэк в формате JSON.""")
        ])

        chain = prompt | self.client.get_llm(max_tokens=3000) | StrOutputParser()

        try:
            response = chain.invoke({"dialog": dialog})

            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response

            feedback_dict = json.loads(json_str)
            return FinalFeedback(**feedback_dict)
        except Exception as e:
            print(f"Ошибка парсинга фидбэка: {e}")
            print(f"Ответ модели: {response if 'response' in locals() else 'нет ответа'}")
            return FinalFeedback(
                grade="Junior",
                hiring_recommendation="No Hire",
                confidence_score=50,
                confirmed_skills=[],
                knowledge_gaps=[],
                soft_skills={"clarity": "N/A", "honesty": "N/A", "engagement": "N/A"},
                roadmap=[]
            )

## Главная система Interview Coach


In [414]:
class InterviewCoachLangChain:
    def __init__(self, gigachat_api_key: str):
        self.client = GigaChatClient(gigachat_api_key)
        self.observer = ObserverAgent(self.client)
        self.interviewer = InterviewerAgent(self.client)
        self.manager = ManagerAgent(self.client)

        self.participant_name = "Оганесян Альберт Самвелович"
        self.position = ""
        self.grade = ""
        self.experience = ""
        self.turns: List[Turn] = []
        self.turn_counter = 0
        self.is_finished = False

    def start_interview(self, name: str, position: str, grade: str, experience: str) -> str:
        self.position = position
        self.grade = grade
        self.experience = experience
        self.turns = []
        self.turn_counter = 0
        self.is_finished = False

        greeting = f"""
Интервьюер: Здравствуйте! Меня зовут AI Interview Coach.

Я буду проводить с вами техническое интервью на позицию {position} ({grade} уровень).

Пожалуйста, представьтесь и кратко расскажите о своем опыте."""

        return greeting

    def process_turn(self, user_message: str) -> str:
        if self.is_finished:
            return "Интервью завершено."

        if "стоп" in user_message.lower() and ("игра" in user_message.lower() or "интервью" in user_message.lower()):
            self.is_finished = True
            return "Интервью завершено. Генерирую финальный фидбэк..."

        self.turn_counter += 1

        if self.turn_counter == 1:
            observer_part = f"[Observer]: Кандидат представился. Начинаем техническую часть."
            next_question, interviewer_reasoning = self.interviewer.generate_question(
                self.position, self.grade, self.experience,
                "Задай первый технический вопрос базового уровня",
                self.turns,
                current_answer=user_message
            )
        else:
            last_question = self.turns[-1].agent_visible_message if self.turns else ""

            observer_analysis = self.observer.analyze(last_question, user_message, self.turns)

            observer_part = f"""[Observer]: {observer_analysis['analysis']}
Рекоммендация для интервьювера: {observer_analysis['recommendation']}"""

            self.interviewer.adjust_difficulty(observer_analysis['knowledge_level'])

            next_question, interviewer_reasoning = self.interviewer.generate_question(
                self.position, self.grade, self.experience,
                observer_analysis['recommendation'],
                self.turns,
                current_answer=user_message
            )

        full_internal_thoughts = f"{observer_part}\n[Interviewer]: {interviewer_reasoning}"

        turn = Turn(
            turn_id=self.turn_counter,
            agent_visible_message=next_question,
            user_message=user_message,
            internal_thoughts=full_internal_thoughts
        )
        self.turns.append(turn)

        return next_question

    def get_feedback(self) -> FinalFeedback:
        if not self.is_finished:
            self.is_finished = True

        return self.manager.generate_feedback(
            self.position, self.grade, self.experience, self.turns
        )

    def save_log(self, filename: str = "interview_log.json"):
        log_data = {
            "participant_name": self.participant_name,
            "turns": [asdict(turn) for turn in self.turns],
            "final_feedback": asdict(self.get_feedback()) if self.is_finished else None
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(log_data, f, ensure_ascii=False, indent=2)

        return filename

## Запуск интервью

In [415]:
coach = InterviewCoachLangChain(GIGACHAT_API_KEY)

greeting = coach.start_interview(
    name="Виктор",
    position="Solution Architect",
    grade="Lead / Expert",
    experience="Заявлено 15 лет опыта, проектировал системы для банков"
)

print(greeting)



Интервьюер: Здравствуйте! Меня зовут AI Interview Coach.

Я буду проводить с вами техническое интервью на позицию Solution Architect (Lead / Expert уровень).

Пожалуйста, представьтесь и кратко расскажите о своем опыте.


In [417]:

while (not coach.is_finished):
    response = input("Ваш ответ: ")

    next_q = coach.process_turn(response)

    if(len(coach.turns) > 0):
      print(f"\nВНУТРЕННИЕ МЫСЛИ:\n{coach.turns[-1].internal_thoughts}")
      print(f"\n\n Интервьюер:\n{next_q}")




Ваш ответ: Привет. Я Виктор, Lead / Expert Solution Architect. 15 лет в индустрии, специализируюсь на распределенных высоконагруженных системах. Давайте пропустим джуниорские вопросы

ВНУТРЕННИЕ МЫСЛИ:
[Observer]: Кандидат явно пытается пропустить базовые вопросы и сразу перейти к сложным темам, демонстрируя нежелание подробно объяснять основы. Это может указывать либо на уверенность в знаниях, либо на попытку скрыть пробелы.
Рекоммендация для интервьювера: Интервьюеру следует попросить кандидата подробно объяснить разницу между этими двумя типами архитектуры и привести примеры ситуаций, когда одна из них предпочтительнее другой.
[Interviewer]: Кандидат заявил о своем опыте работы с распределенными высоконагруженными системами и выразил нежелание обсуждать "джуниорские" вопросы. Однако он проигнорировал основной вопрос об объяснении разницы между монолитной и микросервисной архитектурой. Это может указывать либо на его уверенность в очевидности этого различия, либо на отсутствие четког

## Финальный фидбэк

In [418]:
feedback = coach.get_feedback()
feedback.print_feedback()

Грейд: Junior
Рекомендация: No Hire
Уверенность в результате: 95%

Подтвержденные навыки: 

Пробелы в знаниях:
  - Монолитная и микросервисная архитектура: Основные концепции и различия
  - Git и системы контроля версий: Использование и важность
clarity: Низкая
honesty: Под вопросом
engagement: Минимальная

Дорожная карта обучения:
  1. Изучение основ архитектуры ПО
  2. Освоение инструментов разработки (Git)
  3. Углубленное изучение микросервисной архитектуры


## Сохранение логов


In [419]:
log_file = coach.save_log("interview_langchain_log.json")

with open(log_file, 'r', encoding='utf-8') as f:
    log_data = json.load(f)

print(json.dumps(log_data, ensure_ascii=False, indent=2))
files.download('interview_langchain_log.json')

{
  "participant_name": "Оганесян Альберт Самвелович",
  "turns": [
    {
      "turn_id": 1,
      "agent_visible_message": "<response>\nВиктор, давайте начнем с простого вопроса. Что такое SOA (Сервисно-ориентированная архитектура) и какие основные принципы лежат в ее основе?",
      "user_message": "Привет. Я Виктор, Lead / Expert Solution Architect. 15 лет в индустрии, специализируюсь на распределенных высоконагруженных системах. Давайте пропустим джуниорские вопросы",
      "internal_thoughts": "[Observer]: Кандидат представился. Начинаем техническую часть.\n[Interviewer]: Поскольку наблюдатель рекомендовал начать с базового технического вопроса, я выберу простой вопрос, который позволит проверить базовые знания Виктора как Solution Architect, учитывая его опыт работы с распределенными системами. Для начала подойдет вопрос, касающийся архитектурных принципов или базовых понятий проектирования систем."
    },
    {
      "turn_id": 2,
      "agent_visible_message": "Понял ваше жела

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>